## Live Demo
---

1. Load 3D image.

In [1]:
import ctwidgets

mouse = ctwidgets.load_image("CT140.gff")
print(mouse.data.shape) # dimensions

(290, 211, 429, 1)


2. Extract 3D image (one timepoint).

In [ ]:
mouse_3d = mouse.get_timepoint(0)

3. Launch interactive visualization widget.

In [ ]:
app = ctwidgets.SliceWidget(mouse_3d)

4. Load manual segmentation.

In [ ]:
seg = ctwidgets.load_image("Organs.segff")
print(seg.data.info)
print(seg.metadata)

5. Visualize again, now in color.

In [ ]:
app2 = ctwidgets.SliceWidget(seg.get_timepoint(0), colored=True)

# add class names to colorbar ticks
class_dict = seg.get_class_names()
names = list(class_dict.keys())
indices = list(class_dict.values())
app2.add_class_names(names, indices)

- Pretrained deep learning model for 3D lung tumor segmentation: [nnU-Net](https://github.com/MIC-DKFZ/nnUNet)
    - Published and well-documented.
    - Winner of the 2018 [*Medical Segmentation Decathlon*](http://medicaldecathlon.com/) challenge (public datasets with exper labels).
    
    - Difficulties:
        - Training a model on new data myself is not feasible.
        - Inference is computationally intense.
        - Linux only (!)
        - No validation dataset for challenge (!).

    - Strategy for `ctwidgets`:
        - Segmentations are precomputed.
        - New widget **SegmentationWidget** will offer visualization of dataset + segmentation (+ validation segmentation).
        - If validation segmentation is present, Dice score can be computed.

In [ ]:
# Load nnU-Net results
import os
import numpy as np
import ctwidgets

# find non-empty segmentations
results_dir = "C:/nnUNet/OUTPUT/"
valid_list = []
for file in os.listdir(results_dir):
    if file.endswith(".nii.gz"):
        result = ctwidgets.load_image(results_dir + file)
        if np.max(result.data) > 0:
            valid_list.append(file)            

In [ ]:
seg = ctwidgets.load_image(results_dir + "/" + valid_list[0])
underlay = ctwidgets.load_image("C:/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task006_Lung/imagesTs/lung_002_0000.nii.gz")

print(seg.data.shape)
app3 = ctwidgets.SliceWidget(seg.get_timepoint(), colored=True)
app3.add_class_names(["Unclassified", "Tumor"], [0, 1])

In [ ]:
# In progress: drawing segmentation on top of underlay

seg_coords = np.nonzero(seg.get_timepoint() == 1)
z_const = np.nonzero(seg_coords[2] == 225) # for z=225 slice
app3 = ctwidgets.SliceWidget(underlay.get_timepoint())
app3.widget.add_scatter(x=seg_coords[0][z_const], y=seg_coords[1][z_const], opacity=0.5, mode="markers", marker={"symbol": "square"});